# MCXN947 Indirect chip-specific RTP flow

Indirect flow means that the target is not connecting to the EL2GO service by itself. SPSDK serves a middle-man between the target and EL2GO service, orchestrating the flow.

- obtain UUID from the target
- assign the target to a Device Group in EL2GO (using REST API)
- download Secure Objects from EL2GO into the target
- runs the Provisioning FW to provision the Secure Objects on the target

## 1. Prerequisites
- EdgeLock2GO account and API token
- MCXN947 board
- EL2GO Provisioning Firmware

## 2. Setup

### 2.1 Configuration Setup

First we'll create a configuration file containing all necessary information.
To generate a configuration template use `el2go-host get-template` command

In [1]:
# Initialization cell

from spsdk.utils.jupyter_utils import YamlDiffWidget

# setup this jupyter notebook
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [2]:
YamlDiffWidget("el2go_config.diffc").html

el2go-host get-template -f mcxn947 -o config_template.yaml --force 
The EL2GO template for mcxn947 has been saved into config_template.yaml YAML file


### 2.2 HW Setup

This example uses UART for communication with the target.
You may use `nxpdevscan` utility to list COM ports with devices attached to them.

In [3]:
%! nxpdevscan --port

nxpdevscan --port 
-------- Connected NXP UART Devices --------

Port: COM5
Type: mboot device



## 3. Running EL2GO RTP

To perform the EL2GO RTP in one go, use `el2go-host provision-device` command.

In [4]:
%! el2go-host --verbose provision-device --config config_example.yaml --port com5

el2go-host --verbose provision-device --config config_example.yaml --port com5 
INFO:spsdk.mboot.mcuboot:Connect: identifier='uart', device=com5
INFO:spsdk.mboot.mcuboot:CMD: GetProperty(UniqueDeviceIdent, index=0)
INFO:spsdk.mboot.mcuboot:CMD: Status: 0 (0x0) Success.
INFO:spsdk.mboot.mcuboot:Closing: identifier='uart', device=com5
INFO:spsdk.el2go.client:EL2G-Correlation-ID: dd332b83-0f79-40e8-8475-ff03d93c982d
INFO:spsdk.utils.http_client:Requesting: https://api.qa.sb.edgelock2go.com/api/v1/products/340000274521/device-groups/633/devices
INFO:spsdk.utils.http_client:Response: <Response [202]>
INFO:spsdk.el2go.client:EL2G-Correlation-ID: 5666ac43-bf26-4608-85be-ba27452a71f5
INFO:spsdk.utils.http_client:Requesting: https://api.qa.sb.edgelock2go.com/api/v1/rtp/devices/100779615704702885679946391493172028108/secure-object-provisionings
INFO:spsdk.utils.http_client:Response: <Response [200]>
INFO:spsdk.el2go.client:EL2G-Correlation-ID: 5b15bc7d-1f1f-421c-bc3d-8d90cfd2549d
INFO:spsdk.util

## 4. Quick verification

A full provisioning verification is not possible without a dedicated user application that would test all the Secure Objects. 
Test of RKTH and image encryption keys is done indirectly by loading the user SB3.1 file.

For demonstration purposes, we can read out Secure Object's metadata stored in flash.
The address is defined in the config file as `secure_objects_address` (in our case it's 0x1000).

Each Secure Objects starts with: b"\x40\x0Bedgelock2go"

In [5]:
%! blhost --port com5 read-memory 0x1000 0x10 --use-hexdump

blhost --port com5 read-memory 0x1000 0x10 --use-hexdump 
Reading memory
00000000: 40 0B 65 64 67 65 6C 6F  63 6B 32 67 6F 41 04 00  @.edgelock2goA..
Response status = 0 (0x0) Success.
Response word 1 = 16 (0x10)
Read 16 of 16 bytes.
